In [ ]:
import json
import os
import re

STRING_MATCHING_PATTERN = re.compile(r'([bruf]*)(\"\"\"|\'\'\'|\"|\')(?:(?!\2)(?:\\.|[^\\]))*\2')
                                    
def replace_string_literal(source):
    return re.sub(pattern=STRING_MATCHING_PATTERN, repl='___STR', string=source)

file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/pre_train/java/valid.jsonl'
file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/pre_train/java/test.jsonl'
file_path = '/home/user1-selab3/Documents/research-shradha/CODE-SPT-Code/dataset/pre_train/java/train.jsonl'
output_file_path_txt = 'codetokens-valid.txt'
output_file_path_txt = 'codetokens-test.txt'
output_file_path_txt = 'codetokens-train.txt'

skip_lines = {
    127273, 127274, 127275, 127279, 127280, 127281, 127282, 127283, 127284,
    141780, 141824, 141835, 141847, 141850
}

isGeneralized = False
if isGeneralized == False:
    output_file_path_txt = 'codetokens-valid-nongen.txt'
    output_file_path_txt = 'codetokens-test-nongen.txt'
    output_file_path_txt = 'codetokens-train-nongen.txt'

def count_lines_in_file(filepath):
    with open(filepath, 'r') as file:
        line_count = sum(1 for line in file)
        return line_count

def read_json_get_value(file_path, skip_lines):
    codes = []
    counter_problem_quote = counter_after_skip = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file, start=1):
            if i not in skip_lines:
                counter_after_skip += 1
                data = json.loads(line.strip())
                code = (' '.join(data.get('code_tokens', [])))

                # check 1: count double or single quotes
                double_quote_count = single_quote_count = 0
                double_quote_count += line.count('"')
                single_quote_count += line.count("'")
                if double_quote_count % 2 != 0 or single_quote_count % 2 != 0:
                    # print("Inconsistent quotes detected in the file.")
                    counter_problem_quote += 1
                    continue
                if isGeneralized == True:
                    code = replace_string_literal(code)
                codes.append(code)
    print(f'# of problem_quote: {counter_problem_quote}, counter_after_skip: {counter_after_skip}')
    return codes

def check_data_savedfile(src_file_path, skip_lines):
    codes = read_json_get_value(src_file_path, skip_lines)

    with open(output_file_path_txt, 'w', encoding='utf-8') as txt_file:
        for idx, code in enumerate(codes):
            txt_file.write(f"{code}\n")

    print(f"Saved {output_file_path_txt}, file's lines: {count_lines_in_file(output_file_path_txt)}, data len: {len(codes)}.")

print("Case 1: with problems")
check_data_savedfile(file_path, [])
print("Case 2: removed problems")
check_data_savedfile(file_path, skip_lines)